In [ ]:
# Copyright (c) Roman Lutz. All rights reserved.
# The use and distribution terms for this software are covered by the
# Eclipse Public License 1.0 (http://opensource.org/licenses/eclipse-1.0.php)
# which can be found in the file LICENSE.md at the root of this distribution.
# By using this software in any fashion, you are agreeing to be bound by
# the terms of this license.
# You must not remove this notice, or any other, from this software.

import pdb
import numpy as np
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import feature_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error
from create_datasets import test_players
import time
from metrics import mean_relative_error
from plots import histogram
import sklearn.model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import sklearn




"""
training_results.write('\n')
    training_results.write("BEST")
    training_results.write(str(clf.best_params_))
        score = str(mean_score) + " " + str(params)
        training_results.write(score)
    training_results = open('training_results.txt', 'w')
"""

kernels = ['rbf', 'linear', 'sigmoid']
degrees = [2, 3]
gamma_values = [0.05*k for k in range(1,4)]
C_values = [0.25*k for k in range(1, 5)]
epsilon_values = [0.05*k for k in range(1, 6)]

def hyperparameter(x, y, folds):
    #parameters = {'kernel':('linear', 'rbf', 'sigmoid'), 'degree':(degrees), 'C':(C_values), 'gamma':(gamma_values), 'epsilon':(epsilon_values)}
    #parameters = {'kernel':('linear', 'sigmoid'), 'degree':(degrees), 'C':(C_values), 'gamma':(gamma_values), 'epsilon':(epsilon_values)}
    svr = sklearn.svm.SVR()
    # linear
    linear_kernel = ['linear']
    linear_parameters = {'kernel':(linear_kernel), 'C':(C_values), 'gamma':(gamma_values), 'epsilon':(epsilon_values)}
    clf = sklearn.model_selection.GridSearchCV(svr, linear_parameters, verbose=5, scoring='neg_mean_absolute_error')
    clf.fit(x, y)
    cvres = clf.cv_results_
    print("LINEAR:")
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(mean_score, params)
    print("Best: " + str(clf.best_params_))
    best_linear_param = clf.best_params_
    print(type(clf.best_params_))
    pdb.set_trace()
    """
    # rbf
    rbf_kernel = ['rbf']
    rbf_parameters = {'kernel':(rbf_kernel), 'C':(C_values), 'gamma':(gamma_values), 'epsilon':(epsilon_values)}
    clf = sklearn.model_selection.GridSearchCV(svr, rbf_parameters, verbose=5, scoring='neg_mean_absolute_error')
    clf.fit(x, y)
    cvres = clf.cv_results_
    print("RBF:")
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(mean_score, params)
    print("Best: " + str(clf.best_params_))
    pdb.set_trace()
    # sigmoid
    # poly
    """







# Only one of the feature selection methods can be chosen
FEATURE_SELECTION = False
MANUAL_FEATURE_SELECTION = False
FEATURE_NORMALIZATION = False
HYPERPARAMETER_SELECTION = True
HISTOGRAM = True


"""
train = np.load('train.npy')
test = np.load('test.npy')

train_x = train[:, 2:36].astype(np.float)
train_y = train[:, 36].astype(np.float)
test_x = test[:, 2:36].astype(np.float)
test_y = test[:, 36].astype(np.float)
pdb.set_trace()
"""
"""
# load data
# indices are
# 0: QB id
# 1: QB name
# 2: QB age
# 3: QB years pro
# 4
# 5-16: last game QB stats
# 17-28: last 10 games QB stats
# 29-32: last game defense stats
# 33-36: last 10 games defense stats
# 37: actual fantasy score = target
"""

train = np.load('train.npy')
test = np.load('test.npy')

train_x = train[:, 2:37].astype(np.float)
train_y = train[:, 37].astype(np.float)
test_x = test[:, 2:37].astype(np.float)
test_y = test[:, 37].astype(np.float)



#kernels = ['rbf', 'linear', 'sigmoid', 'poly']
kernels = ['rbf', 'linear', 'sigmoid']
degrees = [2, 3]
gamma_values = [0.05*k for k in range(1,4)]
C_values = [0.25*k for k in range(1, 5)]
epsilon_values = [0.05*k for k in range(1, 6)]

# Feature Normalization
if FEATURE_NORMALIZATION:
    print 'started feature normalization', time.time()
    x = np.concatenate((train_x, test_x), axis=0)
    x = preprocessing.scale(x)
    train_x = x[:len(train_x)]
    test_x = x[len(train_x):]


# Recursive Feature Elimination with cross-validation (RFECV)
if FEATURE_SELECTION:
    print 'started feature selection', time.time()
    selector = feature_selection.RFECV(estimator=SVR(kernel='linear'), step=3, cv=5)
    selector.fit(train_x, train_y)
    train_x = selector.transform(train_x)
    test_x = selector.transform(test_x)
    print selector.ranking_
elif MANUAL_FEATURE_SELECTION: # leave out the two point attempts
    manual_indices = [0, 1, 2, 3, 4, 5, 8, 9, 10, 13, 14, 15, 16, 17, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 32, 33]
    train_x = train_x[:, manual_indices]
    test_x = test_x[:, manual_indices]

# USE NORMAL KFOLD
# hyperparameter selection
if HYPERPARAMETER_SELECTION:
    hyperparameter(train_x, train_y, 5)



/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


TRAIN DATA
TEST DATA
Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] epsilon=0.05, C=0.25, gamma=0.05, kernel=linear .................
[CV]  epsilon=0.05, C=0.25, gamma=0.05, kernel=linear, score=-5.93122702429, total=  33.7s
[CV] epsilon=0.05, C=0.25, gamma=0.05, kernel=linear .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   33.9s remaining:    0.0s


[CV]  epsilon=0.05, C=0.25, gamma=0.05, kernel=linear, score=-6.03511435313, total=  24.2s
[CV] epsilon=0.05, C=0.25, gamma=0.05, kernel=linear .................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   58.4s remaining:    0.0s


...

[CV]  epsilon=0.1, C=0.5, gamma=0.05, kernel=linear, score=-6.03598023119, total=  49.2s
[CV] epsilon=0.1, C=0.5, gamma=0.05, kernel=linear ...................
[CV]  epsilon=0.1, C=0.5, gamma=0.05, kernel=linear, score=-6.03299044496, total= 1.0min
[CV] epsilon=0.1, C=0.5, gamma=0.1, kernel=linear ....................
[CV]  epsilon=0.1, C=0.5, gamma=0.1, kernel=linear, score=-5.92986005344, total=  58.8s
[CV] epsilon=0.1, C=0.5, gamma=0.1, kernel=linear ....................
[CV]  epsilon=0.1, C=0.5, gamma=0.1, kernel=linear, score=-6.03598023119, total=  49.4s
[CV] epsilon=0.1, C=0.5, gamma=0.1, kernel=linear ....................
[CV]  epsilon=0.1, C=0.5, gamma=0.1, kernel=linear, score=-6.03299044496, total= 1.0min
[CV] epsilon=0.1, C=0.5, gamma=0.15, kernel=linear ...................
[CV]  epsilon=0.1, C=0.5, gamma=0.15, kernel=linear, score=-5.92986005344, total=  57.6s
[CV] epsilon=0.1, C=0.5, gamma=0.15, kernel=linear ...................
[CV]  epsilon=0.1, C=0.5, gamma=0.15, kerne

[CV]  epsilon=0.1, C=0.75, gamma=0.15, kernel=linear, score=-6.03125146172, total= 1.5min
[CV] epsilon=0.15, C=0.75, gamma=0.05, kernel=linear .................
[CV]  epsilon=0.15, C=0.75, gamma=0.05, kernel=linear, score=-5.93489248094, total= 1.3min
[CV] epsilon=0.15, C=0.75, gamma=0.05, kernel=linear .................
[CV]  epsilon=0.15, C=0.75, gamma=0.05, kernel=linear, score=-6.03945455348, total= 1.5min
[CV] epsilon=0.15, C=0.75, gamma=0.05, kernel=linear .................
[CV]  epsilon=0.15, C=0.75, gamma=0.05, kernel=linear, score=-6.03296892069, total= 1.5min
[CV] epsilon=0.15, C=0.75, gamma=0.1, kernel=linear ..................
[CV]  epsilon=0.15, C=0.75, gamma=0.1, kernel=linear, score=-5.93489248094, total= 1.3min
[CV] epsilon=0.15, C=0.75, gamma=0.1, kernel=linear ..................
[CV]  epsilon=0.15, C=0.75, gamma=0.1, kernel=linear, score=-6.03945455348, total= 1.5min
[CV] epsilon=0.15, C=0.75, gamma=0.1, kernel=linear ..................
[CV]  epsilon=0.15, C=0.75, gam

[CV]  epsilon=0.15, C=1.0, gamma=0.1, kernel=linear, score=-6.03226111885, total= 2.0min
[CV] epsilon=0.15, C=1.0, gamma=0.15, kernel=linear ..................
[CV]  epsilon=0.15, C=1.0, gamma=0.15, kernel=linear, score=-5.93466517561, total= 1.6min
[CV] epsilon=0.15, C=1.0, gamma=0.15, kernel=linear ..................
[CV]  epsilon=0.15, C=1.0, gamma=0.15, kernel=linear, score=-6.03638651731, total= 2.0min
[CV] epsilon=0.15, C=1.0, gamma=0.15, kernel=linear ..................
[CV]  epsilon=0.15, C=1.0, gamma=0.15, kernel=linear, score=-6.03226111885, total= 2.0min
[CV] epsilon=0.2, C=1.0, gamma=0.05, kernel=linear ...................
[CV]  epsilon=0.2, C=1.0, gamma=0.05, kernel=linear, score=-5.93664994791, total= 1.9min
[CV] epsilon=0.2, C=1.0, gamma=0.05, kernel=linear ...................
[CV]  epsilon=0.2, C=1.0, gamma=0.05, kernel=linear, score=-6.03860911757, total= 2.7min
[CV] epsilon=0.2, C=1.0, gamma=0.05, kernel=linear ...................
[CV]  epsilon=0.2, C=1.0, gamma=0.05,

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed: 227.4min finished


LINEAR:
(-5.998518974068692, {'epsilon': 0.05, 'C': 0.25, 'gamma': 0.05, 'kernel': 'linear'})
(-5.998518974068692, {'epsilon': 0.05, 'C': 0.25, 'gamma': 0.1, 'kernel': 'linear'})
(-5.998518974068692, {'epsilon': 0.05, 'C': 0.25, 'gamma': 0.15000000000000002, 'kernel': 'linear'})
(-5.999550804473324, {'epsilon': 0.1, 'C': 0.25, 'gamma': 0.05, 'kernel': 'linear'})
(-5.999550804473324, {'epsilon': 0.1, 'C': 0.25, 'gamma': 0.1, 'kernel': 'linear'})
(-5.999550804473324, {'epsilon': 0.1, 'C': 0.25, 'gamma': 0.15000000000000002, 'kernel': 'linear'})
(-6.001401880462248, {'epsilon': 0.15000000000000002, 'C': 0.25, 'gamma': 0.05, 'kernel': 'linear'})
(-6.001401880462248, {'epsilon': 0.15000000000000002, 'C': 0.25, 'gamma': 0.1, 'kernel': 'linear'})
(-6.001401880462248, {'epsilon': 0.15000000000000002, 'C': 0.25, 'gamma': 0.15000000000000002, 'kernel': 'linear'})
(-5.999791051257764, {'epsilon': 0.2, 'C': 0.25, 'gamma': 0.05, 'kernel': 'linear'})
(-5.999791051257764, {'epsilon': 0.2, 'C': 0.25, 